In [1]:
from skimage.exposure import rescale_intensity
import numpy as np
import cv2

def convolve(image, kernel, padding, stride):
   
    (iH, iW) = image.shape[:2]
    (kH, kW) = kernel.shape[:2]
    print(iH)
    print(iW)
    image = cv2.copyMakeBorder(image, padding, padding, padding, padding, cv2.BORDER_CONSTANT)
    cv2.imshow('img',image)
    oH = int(((iH + 2 * padding - kH) / stride) + 1 )
    oW = int(((iW + 2 * padding - kW) / stride) + 1 )
    output = np.zeros((oH, oW), dtype="float32")
    for y in np.arange(padding, iH + padding, stride):
        for x in np.arange(padding, iW + padding, stride):
            
            roi = image[y - padding :y +  kH - padding, x - padding :x + kW - padding]
           
            output[int((y - padding) / stride) ,int((x - padding) / stride)] = (roi * kernel).sum()
        
    output = rescale_intensity(output, in_range=(0, 255))
    output = (output * 255).astype("uint8")
 
    
    return output

 
# construct a sharpening filter
sharpen = np.array((
    [0, -1, 0],
    [-1, 5, -1],
    [0, -1, 0]), dtype="int")
# construct the Laplacian kernel used to detect edge-like
# regions of an image
laplacian = np.array((
    [0, 1, 0],
    [1, -4, 1],
    [0, 1, 0]), dtype="int")
 
# construct the Sobel x-axis kernel
sobelX = np.array((
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]), dtype="int")
 
# construct the Sobel y-axis kernel
sobelY = np.array((
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]), dtype="int")
# construct the kernel bank, a list of kernels we're going
# to apply using both our custom `convole` function and
# OpenCV's `filter2D` function
kernelBank = (
    ("sharpen", sharpen),
    ("laplacian", laplacian),
    ("sobel_x", sobelX),
    ("sobel_y", sobelY))
    # load the input image and convert it to grayscale
    
image = cv2.imread('iu.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  


# loop over the kernels
for (kernelName, kernel) in kernelBank:
    # apply the kernel to the grayscale image using both
    # our custom `convole` function and OpenCV's `filter2D`
    # function
    print("[INFO] applying {} kernel".format(kernelName))
    convoleOutput = convolve(gray, kernel, 5, 3)
    opencvOutput = cv2.filter2D(gray, -1, kernel)
    
    # show the output images
    cv2.imshow("original", gray)
    cv2.imshow("{} - convole".format(kernelName), convoleOutput)
    cv2.imshow("{} - opencv".format(kernelName), opencvOutput)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
       
    

error: OpenCV(3.4.1) /io/opencv/modules/imgproc/src/color.cpp:11147: error: (-215) scn == 3 || scn == 4 in function cvtColor
